In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import torch
from ultralytics import settings
from ultralytics.utils.plotting import Annotator

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'background subtraction.ipynb',
 'Dense optical flow cuda.ipynb',
 'Dense optical flow.ipynb',
 'frame difference.ipynb',
 'images',
 'learning.ipynb',
 'lucas kanade optical flow.ipynb',
 'motion_detection_0.ipynb',
 'motion_detection_geeks for feeks.ipynb',
 'motion_detection_learning.ipynb',
 'non max supression.ipynb',
 'opencv_cuda-Copy1.ipynb',
 'opencv_cuda.ipynb',
 'optical flow 2.ipynb',
 'optical flow.ipynb',
 'pt models',
 'video_footages',
 'yolov8 detection.ipynb']

In [3]:
model_files = glob.glob('pt models/*')
model_files

['pt models\\best.pt', 'pt models\\best.torchscript']

In [4]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [5]:
model = YOLO(model_files[0])

In [6]:
model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [7]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\4K Road traffic video for object detection and tracking - free download now!.mp4',
 'video_footages\\Creatures of the Night - trail cam videos.mp4',
 'video_footages\\Dutch bike rush hour this morning was smooth as ever.mp4',
 'video_footages\\Free City Street Footage - Royalty Free Stock Footage - People Walking Stock Footage No Copyright.mp4',
 'video_footages\\morning bike traffic during commute to Dutch university.mp4',
 'video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Road traffic video for object recognition.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\Traffic Flow In The Highway - 4K Stock Videos _ NoCopyright _ AllVideoFree.mp4',
 'video_footages\\Unbelievably busy bicycle crossing in Amsterdam.mp4',
 'video_footages\\Utrecht Centraal Station Bike Pa

In [8]:
labels = model.names

In [9]:
len(labels)

80

In [10]:
cc = torch.randint(0, 255, size=(len(labels), 3)).to(device)

In [11]:
cc[0] = torch.tensor([0, 0, 255], dtype=torch.long)

In [12]:
detect_classes = set(["person", "bicycle", "car", "motorcycle", "airplane",
                      "bus", "train", "truck", "boat", "bird", "cat", "dog",
                      "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe"])

In [15]:
cap = cv2.VideoCapture(video_files[1])
ret, frame = cap.read()
h, w, l = frame.shape
n_h = int(h/1.5)
n_w = int(w/1.5)

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.resize(frame, (n_w, n_h))
        results = model.predict(frame, verbose=False)

        for r in results:
            annotator = Annotator(frame)
            
            boxes = r.boxes
            for box in boxes:
                b = box.xyxy[0]
                c = box.cls
                if labels[int(c)] in detect_classes:
                    annotator.box_label(b, labels[int(c)], color=(int(cc[int(c)][0]),
                                                              int(cc[int(c)][1]),
                                                              int(cc[int(c)][2])))

        img = annotator.result()  
        cv2.imshow('YOLO V8 Detection', img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        

    else:
        break

cap.release()

cv2.destroyAllWindows()

In [ ]:
torch.nn.